In [1]:
#모델 정의
import keras
from keras.models import Model
from keras.layers.core import Dense,Dropout,Flatten
from keras.optimizers import SGD

#
img_row_size, img_col_size = 224, 224
def get_model():
    # 최상위 전결층을 제외한 모델을 불러온다
    
    base_model=keras.applications.vgg16.VGG16(include_top=False,weights=None,
                                             input_shape=(224,224,3))



    # 최상위 전결층을 정의한다
    out = Flatten()(base_model.output)
    out = Dense(2048, activation='relu')(out)
    out = Dropout(0.5)(out)
    out = Dense(2048, activation='relu')(out)
    out = Dropout(0.5)(out)
    output = Dense(10, activation='softmax')(out)
    model = Model(inputs=base_model.input, outputs=output)

    # SGD Optimizer를 사용하여, 모델을 compile한다
    sgd = SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [2]:
#데이터 전처리
#train_path=주소 train주소만 넣어주시면 됩니다
#test_path주소 datagenerator를 하기위해서 test2라는 폴더를 만들어서 안에다가 test이미지를 넣어준다.그래서 test2가되는겁니다.
train_path='C:\\Users\\juhee\\.kaggle\\competitions\\state-farm-distracted-driver-detection\\imgs\\train'
test_path='C:\\Users\\juhee\\.kaggle\\competitions\\state-farm-distracted-driver-detection\\imgs\\test2'

from keras.preprocessing.image import ImageDataGenerator

#이미지 데이터 전처리를 수행하는 함수를 정의한다
datagen=ImageDataGenerator()

train_generator=datagen.flow_from_directory(
    directory=train_path,
    target_size=(224,224),
    batch_size=8,
    class_mode='categorical',
    seed=2018

)

test_generator=datagen.flow_from_directory(
    directory=test_path,
    target_size=(224,224),
    batch_size=1,
    class_mode=None,
    shuffle=False

)

Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [ ]:
from glob import glob
import numpy as np
import subprocess
import os
nfolds = 5
labels = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
#랜덤으로 교차검증을 하기위해서 데이터가 들어갈  temp_train_fold와 temp_valid_fold를 만들어주빈다
temp_train_fold='C:\\Users\\juhee\\.kaggle\\competitions\\state-farm-distracted-driver-detection\\imgs\\fold\\temp_train_fold'
temp_valid_fold='C:\\Users\\juhee\\.kaggle\\competitions\\state-farm-distracted-driver-detection\\imgs\\fold\\temp_valid_fold'
nfolds = 5
labels = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
n_class=10

def generate_split():
    def _generate_temp_folder(root_path):
        os.mkdir(root_path)
        print(root_path)
        for i in range(n_class):
            os.mkdir('{}/c{}'.format(root_path,i))
            path = ('{}/c{}'.format(root_path,i))
            print(path)
    _generate_temp_folder(temp_train_fold)
    _generate_temp_folder(temp_valid_fold)
    train_samples = 0
    valid_samples = 0
    for label in labels:
        files=glob('{}/{}/*jpg'.format(train_path,label))
        for fl in files:
                cmd='cp {} {}/{}/{}'
                if np.random.randint(nfolds)!=1:
                    cmd=cmd.format(fl,temp_train_fold,label,os.path.basename(fl))
                    train_samples += 1
                else:
                    cmd=cmd.format(fl,temp_valid_fold,label,os.path.basename(fl))
                    valid_samples += 1
                subprocess.call(cmd ,stderr=subprocess.STDOUT, shell=True)

    print('# {} train samples | {} valid samples'.format(train_samples, valid_samples))
    return train_samples, valid_samples

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
import pandas as pd
import shutil
from glob import glob
import numpy as np
import subprocess
import os
nfolds = 5
labels = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
n_class=10
print('#Train Model')

datagen=ImageDataGenerator()
test_generator =datagen.flow_from_directory(
    #밑에부분은 test path 집어넣어주시면 됩니다
    'C:\\Users\\juhee\\.kaggle\\competitions\\state-farm-distracted-driver-detection\\imgs\\test2',
    target_size=(224,224),
    batch_size=1,
    class_mode=None,
    shuffle=False)


test_id=[os.path.basename(fl) for fl in glob('{}/imgs/*.jpg'.format(test_path))]

for fold in range(nfolds):
    model= get_model()
    
    train_samples ,valid_samples =generate_split()
    
    train_generator =datagen.flow_from_directory(
    directory=temp_train_fold,
    target_size=(224,224),
    batch_size=1,
    class_mode='categorical',
    seed=10)
    
    valid_generator =datagen.flow_from_directory(
    directory=temp_valid_fold,
    target_size=(224,224),
    batch_size=1,
    class_mode='categorical',
    seed=10)
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
                  ModelCheckpoint('5.h', monitor='val_loss', save_best_only=True, verbose=0)]
    model.fit_generator(train_generator,steps_per_epoch=2236,epochs=500,validation_data=valid_generator,validation_steps=2236,shuffle=True,
                       callbacks=callbacks,verbose=1)


#Train Model
Found 79726 images belonging to 1 classes.
Found 18005 images belonging to 10 classes.
Found 4419 images belonging to 10 classes.
Epoch 1/500
2236/2236 [==============================] - 6033s 3s/step - loss: 2.3087 - acc: 0.1140 - val_loss: 2.2992 - val_acc: 0.1105
Epoch 2/500
2236/2236 [==============================] - 6028s 3s/step - loss: 2.2985 - acc: 0.1123 - val_loss: 2.2995 - val_acc: 0.1073
Epoch 3/500
2236/2236 [==============================] - 27032s 12s/step - loss: 2.2983 - acc: 0.0984 - val_loss: 2.2892 - val_acc: 0.1682
Epoch 4/500
2236/2236 [==============================] - 9595s 4s/step - loss: 2.1463 - acc: 0.2097 - val_loss: 1.7954 - val_acc: 0.3900
Epoch 5/500
2236/2236 [==============================] - 6123s 3s/step - loss: 1.5832 - acc: 0.4302 - val_loss: 1.2670 - val_acc: 0.5322
Epoch 6/500
2236/2236 [==============================] - 7095s 3s/step - loss: 1.2896 - acc: 0.5434 - val_loss: 1.0971 - val_acc: 0.5970
Epoch 7/500
2236/2236 [==========

NameError: name 'model' is not defined